In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
       
def calcSPL(coefs,successes ):
    assert len(coefs) == len(successes)
    return np.mean(coefs *successes)

In [2]:
df = pd.read_csv('/home/dmishkin/dev/nav-agents-new/nav-agents/HUMAN_RUN/mp3d_maction_trace.csv')
df.tail(5)

,task,episode,sceneId,tick,px,py,pz,rotation,actions,actionArgs,forces
9294,-,-,-,19,0.0822,0.551,2.2288,3.3966,turnRight,NaN,NaN
9295,-,-,-,20,0.0822,0.551,2.2288,3.2710,turnRight,NaN,NaN
9296,-,-,-,21,0.0822,0.551,2.2288,3.1453,turnRight,NaN,NaN
9297,-,-,-,22,0.0822,0.551,2.2288,3.0196,turnRight,NaN,NaN
9298,-,-,-,23,0.0822,0.551,2.2288,2.8940,reset,NaN,NaN


In [3]:
pxs = df['px'].as_matrix().astype(np.float32)
pzs = df['pz'].as_matrix().astype(np.float32)
times = df['tick'].as_matrix().astype(np.float32)
actions = df['actions'].tolist()
path_lengths = []
successes = []



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
split_idxs =  df['actions'] == "goal"
changeids = df.index[split_idxs].tolist()
#print (len(changeids))
#print (split_idxs)
i0 = 1
path_lengths = []
successes = []
stepss = []
count = 0
for i in changeids[1:]:
    steps = i - i0
    successes.append(steps < 499)
    x = pxs[i0:i]
    z = pzs[i0:i]
    #print (steps)
    #print (x)
    #print (z)
    #print (df['px'][i0:i])
    #print (df['actions'][i0:i])
    #
    #print ("************")
    path = np.sqrt((z[1:] - z[:-1])**2 + (x[1:] - x[:-1])**2).sum()
    #print (path)
    path_lengths.append(path)
    stepss.append(steps)
    i0=i+1
    count+=1
    #if count > 3:
    #    break
print (len(successes))

50


In [5]:
tt =  pd.read_csv('/home/dmishkin/dev/nav-agents-new/nav-agents/CVPR_LOGS_PLAIN/pointgoal_mp3d_m_goals_dist_dir_DFP_Plain_DFP_InformedAgent_42018-11-14--15_10_55/short_stats.csv')

In [6]:
SHORTEST = tt['shortest_path_length'].as_matrix().astype(np.float32)
PL = tt['path_len'].as_matrix().astype(np.float32)
print (len(SHORTEST))
print (SHORTEST[:50])
print (PL[:50])
SHORTEST2 = []
for i in range(5):
    for j in range(10):
        idx = j*5 + i
        SHORTEST2.append(SHORTEST[idx])
        print (idx)
print (path_lengths)
print (SHORTEST2[:50])

100
[ 0.76568544  4.655635    4.6526914  15.849748   19.025484    1.2
  3.8384776  11.44264    27.071068    0.9485281   7.352691   14.949747
  0.24142136 11.259798    7.8012195   6.5627418   3.4899495  12.284062
 25.412489    1.7970563   7.804163    6.7012196   6.952691    7.518377
 19.104162    9.955635    8.042641    7.359798    6.404163   16.215433
  7.5012193  10.601219    1.7414213   7.138478   21.62254     4.797056
 13.176955    1.6         5.407107   13.7426405   4.6071067   4.472792
  6.4183764  12.681118    2.4071069   4.631371    6.0426407   5.579899
 11.235534    2.1071067 ]
[ 0.2         6.051164    2.5860155   7.5411816   2.77812     0.79843754
  4.046178    6.4892926   3.5672677   0.4         8.643729   15.299457
  0.          3.3418827   4.236092    2.2921062   3.0468357   4.083762
  3.3332908   1.5875      5.00783     2.045761    8.438436    2.21775
  5.288842    7.115942    8.333651    2.4962947   2.2338362   4.2023463
  9.335055    3.415936    3.1164062   4.370718    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


0.36896


In [11]:
#SHORTEST2 = np.array(SHORTEST[:50])
path_lengths = np.array(path_lengths)

coefs = SHORTEST2 / np.maximum(SHORTEST2, path_lengths)
print (coefs)
SPL = calcSPL(coefs, np.array(successes))
print (SPL, np.array(successes).astype(np.float32).mean())

[1.         0.20273958 0.46020028 0.53397876 1.         1.
 0.6369979  0.42504644 0.5656124  0.42384642 1.         0.23366559
 1.         0.5676122  0.38624546 0.30146402 1.         0.36812052
 1.         1.         0.44278055 1.         0.00464006 1.
 0.4160563  0.8681872  0.02731168 0.24071401 0.803616   0.47543052
 1.         1.         0.6425537  1.         0.36346787 0.5953495
 1.         0.422944   0.60153615 0.33638257 0.71463865 0.04831704
 1.         0.05185777 1.         1.         1.         0.5574593
 0.44521937 0.06838192]
0.56879926 0.84
